In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from bs4 import BeautifulSoup
import pandas as pd
from fastai.text import *

In [3]:
with open('/content/en-es.tmx',mode='r') as f:
  text = f.read()
  f.close()

In [5]:
path = '/content/drive/My\ Drive/'

In [10]:
print(text[:500])

<?xml version="1.0" encoding="UTF-8" ?>
<tmx version="1.4">
<header creationdate="Sun Mar  4 00:54:42 2018"
          srclang="en"
          adminlang="en"
          o-tmf="unknown"
          segtype="sentence"
          creationtool="Uplug"
          creationtoolversion="unknown"
          datatype="PlainText" />
  <body>
    <tu>
      <tuv xml:lang="en"><seg>RESOLUTION 55/100</seg></tuv>
      <tuv xml:lang="es"><seg>RESOLUCIÓN 55/100</seg></tuv>
    </tu>
    <tu>
      <tuv xml:lang="en"><s


In [24]:
soup = BeautifulSoup(text,'html.parser')
row = soup.find('tu')
print(row)

x = [i for i in row.get_text().split('\n') if i!='']
print(x)

<tu>
<tuv xml:lang="en"><seg>RESOLUTION 55/100</seg></tuv>
<tuv xml:lang="es"><seg>RESOLUCIÓN 55/100</seg></tuv>
</tu>
['RESOLUTION 55/100', 'RESOLUCIÓN 55/100']


In [25]:
text_list = []
rows = soup.find_all('tu')
for row in rows:
  x = [i for i in row.get_text().split('\n') if i!='']
  text_list.append(x)

In [29]:
df = pd.DataFrame(text_list)

In [47]:
df.head()

,en,es
0,RESOLUTION 55/100,RESOLUCIÓN 55/100
1,"Adopted at the 81st plenary meeting, on 4 Dece...","Aprobada en la 81a. sesión plenaria, celebrada..."
2,"In favour: Afghanistan, Algeria, Angola, Antig...","Votos a favor: Afganistán, Angola, Antigua y B..."
3,"Abstentions: Albania, Andorra, Australia, Aust...","Abstenciones: Albania, Alemania, Andorra, Aust..."
4,55/100. Respect for the right to universal fre...,55/100. Respeto del derecho a la libertad univ...


In [36]:
df.columns = ['en','es']

In [45]:
df.drop(8506,axis=0,inplace=True)

In [48]:
df.to_csv('/content/drive/My Drive/translation_data.csv')

In [3]:
df = pd.read_csv('/content/drive/My Drive/translation_data.csv')

### Pre Processing

In [4]:
df['en'] = df['en'].apply(lambda x:x.lower())
df['es'] = df['es'].apply(lambda x:x.lower())

In [10]:
def seq2seq_collate(samples, pad_idx=1, pad_first=True, backwards=False):
  "Padding to make all samples of equal length"
  samples = to_data(samples)
  max_len_x, max_len_y = max([len(s[0]) for s in samples]), max([len(s[1]) for s in samples])
  res_x = torch.zeros(len(samples),max_len_x).long() + pad_idx
  res_y = torch.zeros(len(samples),max_len_y).long() + pad_idx
  if backwards: pad_first = not pad_first
  for i,s in enumerate(samples):
    if pad_first:
      res_x[i,-len(s[0]):], res_y[i,-len(s[1]):] = LongTensor(s[0]) , LongTensor(s[1])
    else:
      res_x[i,:len(s[0]):], res_y[i,:len(s[1]):] = LongTensor(s[0]) , LongTensor(s[1])
  if backwards: res_x,res_y = res_x.flip(1),res_y.flip(1)
  return res_x,res_y


In [11]:
class Seq2SeqDatabunch(TextDataBunch):
  "Create a `TextDataBunch` for training the RNN Classifier"
  @classmethod
  def create(cls, train_ds, valid_ds, test_ds=None, path:PathOrStr='', bs:int=32, val_bs:int=None, pad_idx=1,
             dl_tfms=None, pad_first=False, device:torch.device=None, no_check:bool=False, backwards:bool=False, **dl_kwargs) -> DataBunch:
             "Function that transform the `datasets` in a `DataBunch` for classification. Passes `**dl_kwargs` on to `DataLoader()`"
             datasets = cls._init_ds(train_ds,valid_ds,test_ds)
             val_bs = ifnone(val_bs,bs)
             collate_fn = partial(seq2seq_collate, pad_idx=pad_idx, pad_first=pad_first, backwards=backwards)
             train_sampler = SortishSampler(datasets[0].x, key=lambda t: len(datasets[0][t][0].data), bs=bs//2)
             train_dl = DataLoader(datasets[0], batch_size=bs, sampler=train_sampler, drop_last=True, **dl_kwargs)
             dataloaders = [train_dl]
             for ds in datasets[1:]:
               lengths =  [len(t) for t in ds.x.items]
               sampler = SortSampler(ds.x, key=lengths.__getitem__)
               dataloaders.append(DataLoader(ds, batch_size=val_bs, sampler=sampler, **dl_kwargs))
             return cls(*dataloaders, path=path, device=device, collate_fn=collate_fn, no_check=no_check)


In [12]:
class Seq2SeqTextList(TextList):
  _bunch = Seq2SeqDatabunch
  _label_cls = TextList

In [40]:
src = Seq2SeqTextList.from_df(df,path=path,cols = 'es').split_by_rand_pct(seed=42).label_from_df(cols='en',label_cls=TextList)

In [41]:
np.percentile([len(o) for o in src.train.x.items] + [len(o) for o in src.valid.x.items], 90)

114.0

In [42]:
np.percentile([len(o) for o in src.train.y.items] + [len(o) for o in src.valid.y.items], 90)

101.0

In [43]:
src = src.filter_by_func(lambda x,y:len(x)>120 or len(y)>110)

In [44]:
len(src.train + src.valid)

7763

In [45]:
data = src.databunch()

In [46]:
data.save()

In [49]:
data

Seq2SeqDatabunch;

Train: LabelList (6221 items)
x: Seq2SeqTextList
xxbos resolución 55 / 100,xxbos aprobada en la 81a . sesión plenaria , celebrada el 4 de diciembre de 2000 , por recomendación de la comisión ( a / 55 / 602 / add.2 , párr . 94),el proyecto de resolución recomendado en el informe fue patrocinado en la comisión por los países siguientes : bolivia , cuba , el salvador , ghana y honduras . en votación registrada de 106 votos contra uno y 67 abstenciones , como sigue :,xxbos 55 / 100 . respeto del derecho a la libertad universal de xxunk e importancia vital de la reunificación de las familias,xxbos recordando las disposiciones de la declaración universal de derechos humanosresolución 217 a ( iii ) . , y el artículo 12 del pacto internacional de derechos civiles y políticosvéase resolución 2200 a ( xxi ) , anexo . ,,xxbos subrayando que , conforme se declara en el programa de acción de la conferencia internacional sobre la población y el desarrolloinforme de la conferencia 

In [13]:
data = load_data('/content/drive/My Drive')

In [14]:
data.show_batch()

text,target
"xxbos 7 . alienta al fondo común para los productos básicos a que , en cooperación con el centro de comercio internacional , la conferencia de las naciones unidas sobre comercio y desarrollo y otros órganos pertinentes , siga ampliando las actividades de la segunda cuenta prestando en forma efectiva el apoyo necesario para investigación y desarrollo y servicios de xxunk en los países en desarrollo , lo que incluye","xxbos 7 . encourages the common fund for commodities , in cooperation with the international trade centre , the united nations conference on trade and development and other relevant bodies , to continue to expand the activities of its second account with the necessary and effective support for research and development and extension services in developing countries , including xxunk research on production and processing aimed at xxunk and small - sized and medium - sized enterprises in developing countries , in order to widen the scope of activities and thus ensure the effective participation of all stakeholders ;"
"xxbos 12 . insta a los estados a que adopten medidas especiales para proteger , en particular a las niñas afectadas por la guerra de las enfermedades de transmisión sexual , como el virus de la inmunodeficiencia humana y el síndrome de inmunodeficiencia adquirida y de la violencia que tiene su origen en el género , como la violación y el abuso sexual , la tortura , la explotación sexual","xxbos 12 . urges states to take special measures for the protection of war - affected girls and in particular to protect them from sexually transmitted diseases , such as human immunodeficiency virus / acquired immunodeficiency syndrome , and gender - based violence , including rape and sexual abuse , torture , sexual exploitation , abduction and forced labour , paying special attention to refugee and displaced girls , and to take into account the special needs of the war - affected girl child in the delivery of humanitarian assistance and disarmament , demobilization and reintegration processes ;"
"xxbos 2 . observa y pide que se amplíen y fortalezcan más las actividades que realizan numerosos países africanos con miras a intensificar los progresos en el ámbito de las reformas económicas , lo que incluye la aplicación de políticas macroeconómicas adecuadas ; la promoción del sector privado ; el perfeccionamiento del proceso de democratización ; el fortalecimiento de la sociedad civil ; la gestión de los asuntos públicos con","xxbos 2 . recognizes and calls for further broadening and strengthening of efforts by many african countries to enhance progress in areas covering economic reforms , including the xxunk in place of sound macroeconomic policies , promotion of the private sector , enhancement of the democratization process and strengthening of civil society and participatory , transparent and accountable governance and the rule of law , as well as increased attention to the human dimension , especially education , gender , population , health and south - south cooperation ;"
"xxbos a ) cooperen y realicen actividades concertadas con miras a reducir las disparidades existentes en las corrientes de información a todos los niveles incrementando la asistencia para el desarrollo de las capacidades e infraestructuras de comunicaciones de los países en desarrollo , teniendo debidamente en cuenta sus necesidades y las prioridades que esos países xxunk a esas esferas , para que tanto ellos como sus medios de información públicos","xxbos ( a ) to cooperate and xxunk with a view to reducing existing disparities in information flows at all levels by increasing assistance for the development of communication infrastructures and capabilities in developing countries , with due regard for their needs and the priorities attached to such areas by those countries , and in order to enable them and the public , private or other media in developing countries to develop their own information and communication policies freely 

### Model Creation

In [52]:
# Downloading Embeddings from fastext
!pip install fasttext

     |████████████████████████████████| 71kB 2.0MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3019416 sha256=4e2d608a26b5a0fff7a85b454ea3c4635c987834fcf622b4af1cf456922d572e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [20]:
from fasttext import FastText as ft

In [56]:
# Downloading Pretrained Word Embedding for both languages
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz -P {path}
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz -P {path}

--2020-06-17 06:37:09--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.en.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4503593528 (4.2G) [application/octet-stream]
Saving to: ‘/content/drive/My Drive/cc.en.300.bin.gz’

cc.en.300.bin.gz    100%[===================>]   4.19G  12.4MB/s    in 5m 49s  

2020-06-17 06:42:58 (12.3 MB/s) - ‘/content/drive/My Drive/cc.en.300.bin.gz’ saved [4503593528/4503593528]

--2020-06-17 06:43:01--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.es.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 172.67.9.4, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 45001076

In [16]:
!gunzip '/content/drive/My Drive/cc.en.300.bin.gz' > '/content/cc.en.300.bin'
!gunzip '/content/drive/My Drive/cc.es.300.bin.gz' >  '/content/cc.es.300.bin'

In [22]:
#es_vecs = ft.load_model('/content/cc.es.300.bin')
#en_vecs = ft.load_model('/content/cc.en.300.bin')